    Roman Peerboom  10791523
    Floris de Vries 11710799

In [85]:
import numpy
import simpy
import random
import statistics 

In [121]:
arrival_rate = 10              # Every ten seconds someone arrives, mean of the interval
time_in_system = 20            # Mean time in system 

t_arr = random.expovariate(1.0 / arrival_rate)
t_serv = random.expovariate(1.0 / time_in_system)

print(t_arr)
print(t_serv)

9.213696518733505
25.634480005729827
